In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

import torch
from torchvision import models

from captum.attr import  GuidedBackprop

# Helper functions
import sys
sys.path.append('../')

from utils.visualise import display_imagenet_output, process_attributions
from utils.datasets import preprocess_imagenet_image

## Load model and sample image

In [ ]:
# Download example image
import sys
sys.path.append('../')

from utils.download import save_image

url = "https://upload.wikimedia.org/wikipedia/commons/4/4b/Israel-2013-Makhtesh_Ramon_02_%28Ibex%29.jpg"
save_image(url, "goat.png")

In [ ]:
# Load a sample image
img_path = "goat.png"
img = Image.open(img_path).convert("RGB")

plt.imshow(img)
plt.title("Input Image")
plt.axis('off')

In [ ]:
# Load the pre-trained model (e.g., ResNet50)
model = models.resnet50(pretrained=True)

# Set the model to gpu
device = torch.device('mps' if torch.backends.mps.is_built()
                      else 'cuda' if torch.cuda.is_available()
                      else 'cpu')
model.to(device)

# Set the model to evaluation mode
model.eval()
model.zero_grad()

In [ ]:
# Preprocess the image
original_img_tensor = preprocess_imagenet_image(img_path)
original_img_tensor = original_img_tensor.to(device)

# Clone tensor to avoid in-place operations
img_tensor = original_img_tensor.clone()
img_tensor.requires_grad_() # Enable gradient tracking

predictions = model(img_tensor)

# Decode the output
display_imagenet_output(predictions,n=5)

## SmoothGrad + Standard Backpropogation

In [ ]:
# Reset gradients
model.zero_grad()

# We will use this class for all gradient computations
target_class = predictions.argmax()

# Compute gradients w.r.t to logit by performing backward pass
predictions[:, target_class].backward()

In [ ]:
# Get the gradients
standard_backprop_grads = img_tensor.grad.detach().cpu().numpy()

grads = standard_backprop_grads[0].copy()
grads = process_attributions(grads, activation="abs",skew= 0.5, colormap="viridis")

In [ ]:
# Parameters
n_samples = 50  # number of noisy samples
noise_sigma = 0.15 * (img_tensor.max()- img_tensor.min()).item() # standard deviation of noise

# SmoothGrad computation
smooth_grads = torch.zeros_like(original_img_tensor)

for i in range(n_samples):
    # Add noise to original image
    noise = torch.randn_like(original_img_tensor) * noise_sigma
    noisy_img = original_img_tensor + noise
    noisy_img.requires_grad_()

    # Reset gradients to be safe
    if noisy_img.grad is not None:
        noisy_img.grad.zero_()

    # Forward pass
    preds = model(noisy_img)
    model.zero_grad()

    # Backward pass
    preds[:, target_class].backward()

    # Get gradients
    noisy_grad = noisy_img.grad

    # Accumulate gradients
    smooth_grads += noisy_grad

smooth_grads /= n_samples

# Convert to numpy
smooth_grads_np = smooth_grads.detach().cpu().numpy()[0].copy()

In [ ]:
# Process attribution map (same as your existing function)
smoothgrad_map = process_attributions(smooth_grads_np, activation="abs", skew=0.5, colormap="viridis")

# Visualization
fig, ax = plt.subplots(1, 3, figsize=(12, 6))
ax[0].imshow(img)
ax[0].set_title("Input Image")
ax[1].imshow(grads)
ax[1].set_title("Gradients")
ax[2].imshow(smoothgrad_map)
ax[2].set_title(f"SmoothGrad + Gradients")

for a in ax:
    a.set_xticks([])
    a.set_yticks([])

## SmoothGrad + Guided Backpropagation

In [ ]:
img_tensor = original_img_tensor.clone()
img_tensor.requires_grad_()

#  Guided Backprop
guided_bp = GuidedBackprop(model)
gb_attr = guided_bp.attribute(img_tensor, target=target_class)

gb_attr = process_attributions(gb_attr, activation="abs", skew=0.5, colormap="viridis")

In [ ]:
n_samples = 50         # number of noisy samples
noise_sigma = 0.15 * (original_img_tensor.max() - original_img_tensor.min()).item() # standard deviation of noise

# SmoothGrad + Guided Backpropagation
smooth_grads_gb = torch.zeros_like(original_img_tensor)

for i in range(n_samples):
    # Add Gaussian noise to original image
    noise = torch.randn_like(original_img_tensor) * noise_sigma
    noisy_img = original_img_tensor + noise
    noisy_img.requires_grad_()

     # Reset gradients to be safe
    if noisy_img.grad is not None:
        noisy_img.grad.zero_()

    # Guided Backpropagation
    guided_bp = GuidedBackprop(model)
    gb_attr_noisy = guided_bp.attribute(noisy_img, target=target_class)

    # Accumulate gradients
    smooth_grads_gb += gb_attr_noisy.detach()

# Average the accumulated gradients
smooth_grads_gb /= n_samples

# Convert to numpy
smooth_grads_gb_np = smooth_grads_gb.detach().cpu().numpy()[0]


# Process attribution map
smoothgrad_gb_map = process_attributions(smooth_grads_gb_np, activation="abs", skew=0.5, colormap="viridis")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 6))

ax[0].imshow(img)
ax[0].set_title("Input Image")

ax[1].imshow(gb_attr)
ax[1].set_title("GBP")

ax[2].imshow(smoothgrad_gb_map)
ax[2].set_title("SmoothGrad + GBP")

for a in ax:
    a.set_xticks([])
    a.set_yticks([])